# n-step Bootstrapping

In this chapter, we unify the MC methods and the one-step TD methods presented in the previous two chapters. Neither MC methods nor one-step TD methods are always the best.
In this chapter, we present **n-step** TD methods that generalize both methods so that one can shift from one to the other smoothly as needed to meet the demands of a particular task.
The best methods are often intermediate between the two extremes.

Another way of looking at the benefits of n-step methods is that they free you from the tyranny of the time step (one time step only). With one-step TD methods the same time step determines how often the action can be changed and the time interval over which bootstrapping is done.
In many applications one wants to be able to update the action very fast to take into account anything that has changed, but bootstrapping works best if it is over a length of time in which a significant and recognizable state change has occurred. With one-step TD methods,
these time intervals are the same, and so a compromise must be made. However, n-step methods enable bootstrapping to occur over multiple steps, freeing us from the tyranny of the single time step.

## N-step TD Prediction

What is the space of methods lying between Monte Carlo and TD methods? Consider
estimating v⇡ from sample episodes generated using ⇡. Monte Carlo methods perform
an update for each state based on the entire sequence of observed rewards from that
state until the end of the episode. The update of one-step TD methods, on the other
hand, is based on just the one next reward, bootstrapping from the value of the state
one step later as a proxy for the remaining rewards. *One kind of intermediate method,
then, would perform an update based on an intermediate number of rewards: more than
one, but less than all of them until termination*. For example, a two-step update would
be based on the first two rewards, and the estimated value of the state two steps later.
Similarly, we could have three-step updates, four-step updates, and so on.

The methods that use n-step updates are still TD methods because they still change an earlier estimate based on how it differs from a later estimate. Now the later estimate is not one step later, but n-step later. Methods in which
the TD extends over n steps are called n-step TD methods. The TD methods introduced in the previous chapter all used on-step updates, which is why we called them one-step TD methods.

More formally, consider the update of the estimated value of state $S_{t}$ as a result of the state reward sequence, $S_{t}, R_{t+1}, S_{t+1}, ..., R_{T}, S_{T}$. We know that in MC updates the estimate of $v_{\pi} (S_{t})$ is updated in the direction of the complete return:

$G_{t} =  \sum_{k=0}^{T} \gamma^k R_{t+k+1}$

Where T is the last time step of the episode. Let us call this quantity the target of the update. Whereas in MC updates the target is the return, in one-step updates, the target is the first reward plus the discounted estimated value of the next state ($\hat{T^{\pi}} V $), we call this
*one-step return (empirical Bellman operator)*

$\hat{T^{\pi}} V = G_{t:t+1} = R_{t+1} + \gamma V_{t} (S_{t+1})$

Where $V_{t}$ here is the estimate at time t of $v_{\pi}$. The subscripts indicate that it is a truncated return for time t using rewards up until time t+1, with the discounted estimate $\gamma V_{t} (S_{t+1})$ taking the place of the rest terms of return ($\gamma R_{t+2} + ... + \gamma^{T-t-1} R_{T}$). Our point is that, this idea makes just as much sense after two steps as it does after one.
The target for a two-step update is the two-step return:

$G_{t:t+2} = R_{t+1} + \gamma R_{t+2} + \gamma^2 V_{t} (S_{t+2})$

Where now $\gamma^2 V_{t+1} (S_{t+2})$ corrects for the absence of the rest of the terms of return. Similarly, the target for an arbitrary n-step update is the n-step return.

$G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + ... + \gamma^n V_{t+n-1}(S_{t+n})$

For all n, t such that $n \geq 1$ and $0 \leq t < T-n$. All n-step returns can be considered approximations to the full return, truncated after n steps and then corrected for the remaining missing terms by $V_{t+n-1} (S_{t+n})$. If $t+n \geq T$, then all the missing terms are taken as zero, and the n-step return defined to be equal to the ordinary full return $G_{t:t+n} = G_{t}$

Note that n-step returns for $n > 1$ involve future rewards and states that are not available at the time of transition from t to t-1. No real algorithm can use the n-step return until after it has seen R_{t+n} and computed $V_{t+n-1}$. The first time these are available is $t+n$. The natural state-value learning algorithm for using n-step return is thus,

$V_{t+n} (S_t) = V_{t+n-1} (S_{t}) + \alpha [G_{t:t+n} - V_{t+n-1} (S_{t})]$ for $0 \leq t < T$

while the values of all other states remain unchanged: $V_{t+n} (s) = V_{t+n-1} (s), \forall s \neq S_t$. We call this algorithm n-step TD. Note that no changes at all are made during the first n - 1 steps of each episode. To make up for that, an equal number of additional updates are made at the end of the episode, after termination and before starting the next episode.

<img src='pngs/n-step-td.png'>

## N-step Sarsa

The main idea is to simply switch states for actions  (for state action value function) and then use an $\epsilon-$greedy policy.

$G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + ... + \gamma^n Q_{t+n-1}(S_{t+n}, A_{t+n})$

with $G_{t:t+n} = G_{t}$ if $t+n \geq T$. The natural algorithm is then:

$Q_{t+n} (S_t, A_{t}) = Q_{t+n-1} (S_{t}, A_{t}) + \alpha [G_{t:t+n} - Q_{t+n-1} (S_{t}, A_{t})]$ for $0 \leq t < T$

While the values of all other states remain unchanged: $Q_{t+n} (s, a) = Q_{t+n+1} (s, a), \forall s, a$ s.t $s \neq S_{t}$ or $a \neq A_{t}$

<img src='pngs/n_step_sarsa.png'>

## N-step Expected Sarsa

The n-step expected Sarsa is the same as n-step Sarsa, except that its last element is a expected value. This algorithm can be described by the same equation as n-step Sarsa except with the n-step return redefined as:

$G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + ... + \gamma^n \bar{Q}_{t+n-1}(S_{t+n})$

Where $\bar{Q}_{t} (s) = \sum_{a} \pi(a | s) Q_{t}(s, a)$, if s is terminal, then its expected approximate value is defined to be 0.


## N-step off-policy learning

Recall that off-policy learning is learning the value function for one policy $\pi$, while following another policy b. Often, $\pi$ is the greedy policy for the current action-value-function estimate, and b is a more exploratory policy,
perhaps $\epsilon$-greedy policy. In order to use the data from b we must take into account the difference between the two policies, using their relative probability of taking the actions that were taken. In n-step methods, returns are constructed over n steps, so we are interested in the relative probability of just those n actions.
For example, to make a simple off-policy version of n-step TD, the update for time t (actually made at time t+n) can simply be weighted by $\rho_{t: t+n-1}$ (n - 1 steps):

$V_{t+n} (S_{t}) = V_{t+n-1} (S_{t}) + \alpha \rho_{t:t+n-1} [G_{t:t+n} - V_{t+n-1} (S_{t})], 0 \leq t < T$

Where $\rho_{t:t+n-1}$, called the **importance sampling ratio**, is the relative probability under the two policies of taking the n actions from $A_{t}$ to $A_{t+n-1}$

$\rho_{t:h} = \prod_{k=t}^{min(h, T-1)} \frac{\pi(A_k | S_k)}{b(A_k | S_k)}$

For example, if any one of the actions would never be taken by $\pi$, then the n-step return should given zero weight and be totally ignored. On the other hand, if by chance an action is taken that $\pi$ would take with much greater probability than b does, then this will increase the weight that would otherwise be given to the return.
This makes sense because that action is characteristic of $\pi$ but is selected only rarely by b and thus rarely appears in the data. To make up for this we have to over-weight it when it does occur. note that if the two policies are actually the same, then the importance sampling ratio is always 1. Thus our new update generalizes and can completely replace our earlier n-step TD update.

**$Q_{t+n} (S_t, A_t) = Q_{t+n-1} (S_t, A_t) + \alpha \rho_{t+1:t+n} [G_{t: t+n} - Q_{t+n-1} (S_{t}, A_{t})]$**

for $0 \leq t < T$. Note that the importance sampling ratio here starts and ends one step later than for n-step TD. This is because here we are updating a state-action pair. We do not have to care how likely we were to select the action; now that we have selected it we want to learn fully from what happens, with importance sampling only for subsequent actions.

The off-policy version of n-step Expected Sarsa would use the same update as above for n-step Sarsa except that the importance sampling ratio would have one less factor in it. That is, the above equation could use $\rho_{t+1: t+n-1}$ instead of $\rho_{t+1: t+n}$ and of course it would use the expected Sarsa version of the n-step return. This is because in expected Sarsa, all possible actions are taken into account in the last state; the one actually taken has no effect and does not have to be corrected for.

## Off-policy learning Without Importance Sampling: The n-step Tree Backup Algorithm

Is off-policy learning possible without importance sampling? Q-learning and Expected Sarsa do this for the one-step case, but is there a corresponding multi-step algorithm? The answer is yes, the algorithm is called **tree-backup algorithm**.

Equations for the n-step tree-backup algorithm:

One Step is the same as expected Sarsa:

$G_{t:t+1} = R_{t+1} + \gamma \sum_{a} \pi(a | S_{t+1}) Q_{t} (S_{t+1}, a)$

For $t < T-1$, and the two-step tree-backup return is:

$G_{t:t+2} = R_{t+1} + \gamma \underbrace{\sum_{a\neq A_{t+1}} \pi(a | S_{t+1}) Q_{t+1} (S_{t+1}, a)}_{\text{expected value of actions not selected}} + \gamma \underbrace{\pi(A_{t+1} | S_{t+1})(R_{t+2} + \gamma \sum_{a} \pi(a | S_{t+2}) Q_{t+1} (S_{t+2}, a))}_{\text{future return weighted by selected action}} = R_{t+1} + \gamma \sum_{a\neq A_{t+1}} \pi(a | S_{t+1}) Q_{t+1} (S_{t+1}, a) + \gamma \pi(A_{t+1} | S_{t+1})G_{t+1: t:2}$

General recursive definition of the tree-backup n-step return:

$G_{t: t+n} = R_{t+1} + \gamma \pi(A_{t+1} | S_{t+1}) G_{t+1:t+n} + \gamma \sum_{a\neq A_{t+1}} \pi(a | S_{t+1}) Q_{t+n-1} (S_{t+1}, a) $

For $t < T-1, n \geq 2$, with the n = 1 case handled by $G_{t:t+1}$ except for $G_{T-1: t+n} = R_{T}$. This target is then used with the usual action-value update rule from n-step SARSA:

$Q_{t+n} (S_t, A_{t}) = Q_{t+n-1} (S_{t}, A_{t}) + \alpha [G_{t:t+n} - Q_{t+n-1} (S_{t}, A_{t})], 0 \leq t < T$

<img src="pngs/n-step-tree.png">

## closer look at tree-backup algorithm

https://ai.stackexchange.com/questions/9518/why-is-the-n-step-tree-backup-algorithm-an-off-policy-algorithm

## A Unifying Algorithm: n-step $Q(\theta)$

So far we have seen three different kinds of action-value algorithms, n-step SARSA, expected SARSA, tree-backup. So to waht extend can these algorithms be unified?

One idea for unification is that one might decide on a step-by-step basis whether one wanted to take action as a sample, as in Sarsa, or consider the expectation over all actions instead. Then if one chose always to sample, one would obtain SARSA, whereas if one chose never to sample,
one would get the tree-backup algorithm. Expected Sarsa would be the case where one chose to sample for all steps except for the last one.

